This code is taken from CS287, Advanced Robotics, University of California at Berkeley, taught by Pieter Abbeel.

## LQR for Linear systems

First import the required libraries. Using virtual enviornment is recommended.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.io import loadmat
from scipy.io import savemat
import seaborn as sns
sns.set_style('darkgrid')

We start with a linear system

In [ ]:
A = np.array([[0.0481, -0.5049, 0.0299, 2.6544, 1.0608],
              [2.3846, -0.2312, -0.1260, -0.7945, 0.5279],
              [1.4019, -0.6394, -0.1401, 0.5484, 0.1624],
              [-0.0254, 0.4595, -0.0862, 2.1750, 1.1012],
              [0.5172, 0.5060, 1.6579, -0.9407, -1.4441]])
B = np.array([[-0.7789, -1.2076],
              [0.4299, -1.6041],
              [0.2006, -1.7395],
              [0.8302, 0.2295],
              [-1.8465, 1.2780]])
dx = A.shape[0]
du = B.shape[1]

Check to see if the system is controllable

In [ ]:
# verify the above statement
lst = [B]
"""YOUR CODE HERE"""
for i in range(1, dx-1):
    lst.append(np.zeros((dx, du)))

"""YOUR CODE ENDS HERE"""
np.linalg.matrix_rank(np.hstack(lst))

Implement the value iteration algorithm.

In [ ]:
# implement the infinite horizon optimal feedback controller
def lqr_infinite_horizon(A, B, Q, R):
    """
    find the infinite horizon K and P through running LQR back-ups
    until l2-norm(K_new - K_curr, 2) <= 1e-4
    return: K, P
    """
    
    dx, du = A.shape[0], B.shape[1]
    P, K_current = np.eye(dx), np.zeros((du, dx))
    threshold = 1e-4
    Delta = 10.0
    """YOUR CODE HERE"""
    
    
    """YOUR CODE ENDS HERE"""
    return K_new, P

Let's define our problem. Find the optimal controller with VI and compare it with the exact analtyical solution.

In [ ]:
Q, R = np.eye(dx), np.eye(du)
K_inf, P_inf = lqr_infinite_horizon(A, B, Q, R)
P_exact, K_exact = np.eye(dx), np.zeros((du, dx))
"""YOUR CODE HERE"""

"""YOUR CODE ENDS HERE"""


Code the linear controller and plot the results.

In [ ]:
# fill in the simulation to use your controller, K_inf, at each timestep then run the cell to generate plots
def simulate(A, B, K_inf, n_starting_states, T, noise=None):
    for s in np.arange(n_starting_states):
        x, u = np.zeros((K_inf.shape[1], T+1)), np.zeros((K_inf.shape[0], T+1))
        x[:,0] = starting_states[:,s]
        for t in np.arange(T):
            """YOUR CODE HERE"""
            u[:,t] =
            """YOUR CODE ENDS HERE"""
            x[:,t+1] = A @ x[:,t] + B @ u[:,t]
            if noise is not None:
                x[:,t+1] += noise[:,t]
        plt.plot(x.T, linewidth=.7)
        plt.xlabel('time')
        plt.title("Noisy Linear System Start State #{}".format(s)) if noise is not None else plt.title("Linear System Start State #{}".format(s))
        plt.legend(["dim"+str(i) for i in range(len(x))])
        plt.show()
        
        
starting_states = np.array([[-1.9613, 1.9277, -0.2442],
                            [-1.3127, -0.2406, -0.0260],
                            [0.0698, -0.5860, -0.7522],
                            [0.0935, -0.1524, -0.9680],
                            [1.2494, 0.5397, -0.5146]])
n_starting_states = starting_states.shape[1]
T = 20 # simulating for 20 steps
simulate(A, B, K_inf, n_starting_states, T)

# and in the presence of noise:
noise_id = "p_a_w"
T = 100 # simulating for 100 steps
simulate(A, B, K_inf, n_starting_states, T, noise=loadmat("mats/"+noise_id+".mat")[noise_id])

## LQ for nonlinear systems
For nonlinear systems, we linearize around a given point

In [ ]:
# implement linearization about a point
def linearize_dynamics(f, x_ref, u_ref, dt, my_eps, x_ref_tplus1=None):
    """
    f : dynamics simulator
    my_eps : delta for forward and backward differences you'll need
    NOTE: please use centered finite differences!
    
    x(:,t+1) - x_ref  approximately = A*( x(:,t)-x_ref ) + B* ( u(:,t) - u_ref ) + c
    If we pick x_ref and u_ref to constitute a fixed point, then c == 0 
    
    In this part where we have LQ for nonlinear systems, you do not need to use the optional argument (nor c).
    Later, you'll have to revisit and modify this function 
        --at this point, you'll want to use the optional argument and the resulting c. 
    
    return: A, B, c
    """
    
    if x_ref_tplus1 is not None:
        x_ref_next = x_ref_tplus1
    else:
        x_ref_next = x_ref
    
    dx, du = x_ref.shape[0], u_ref.shape[0]
    A, B = np.zeros((dx, dx)), np.zeros((dx, du))
    
    """YOUR CODE HERE"""
    
    """YOUR CODE ENDS HERE"""
    
    c = f(x_ref, u_ref, dt) - x_ref_next
    if len(B.shape) == 1:
        return A, B.reshape(-1, 1), c
    return A, B, c

take an environment and find the infinite horizon controller for the linearized system

In [ ]:

def lqr_nonlinear(config):
    env = config['env']
    f = config['f']
    dt = 0.1 # we work with discrete time
    my_eps = 0.01 # finite difference for numerical differentiation
    
    # load in our reference points 
    x_ref, u_ref = config['x_ref'], config['u_ref']
    
    # linearize
    A, B, c = linearize_dynamics(f, x_ref, u_ref, dt, my_eps)
    dx, du = A.shape[0], B.shape[1]
    Q, R = np.eye(dx), np.eye(du)*2
    
    # solve for the linearized system
    K_inf, P_inf = lqr_infinite_horizon(A, B, Q, R) # you implemented in part (a)
    
    # recognize the simulation code from part (a)? modify it to use your controller at each timestep
    def simulate(K_inf, f, x_ref, u_ref, dt, n_starting_states, T, noise=None):
        for s in np.arange(n_starting_states):
            x, u = np.zeros((K_inf.shape[1], T+1)), np.zeros((K_inf.shape[0], T+1))
            x[:,0] = starting_states[:,s]
            for t in np.arange(T):
                """YOUR CODE HERE"""
                u[:,t] = 
                """YOUR CODE ENDS HERE"""
                x[:,t+1] = f(x[:,t], u[:,t], dt)
                if "p_val" in config.keys():
                    perturbation_values = config["p_val"]
                    perturb = perturbation_values[t//(T//len(perturbation_values))]
                    x[:,t+1] = f(x[:,t], u[:,t], dt, rollout=True,perturb=perturb)
                if env is not None:
                    if t % 5 == 0:
                        plt.clf()
                        plt.axis('off')
                        plt.grid(b=None)
                        plt.imshow(env.render(mode='rgb_array', width=256, height=256))
                        plt.title("Perturbation Magnitude {}".format(perturb))
                        display.clear_output(wait=True)
                        display.display(plt.gcf())
                
                if noise is not None:
                    x[:,t+1] += noise[:,t]
            if env is not None:
                plt.clf()
        
            plt.plot(x.T[:-1], linewidth=.6)
            plt.plot(np.squeeze(u.T[:-1])/10.0, linewidth=.7, linestyle='--') # scaling for clarity
            if 'legend' in config.keys():
                config['legend'].append('u')
                plt.legend(config['legend'])
            else:
                legend_elements = [Line2D([0], [0], label='x'),Line2D([0], [0], linestyle='--', label='u')]
                plt.legend(handles=legend_elements)
            plt.xlabel('time')
            plt.title(config["exp_name"])
            plt.show()
        
    # now let's simulate and see what happens for a few different starting states
    starting_states = config['ss']
    n_starting_states = starting_states.shape[1]
    T = config['steps'] # simulating for T steps
    simulate(K_inf, f, x_ref, u_ref, dt, n_starting_states, T)
    if 'noise' in config.keys():
        # and now in the presence of noise
        noise_id = config['noise']
        noise_loaded = loadmat("mats/"+noise_id+".mat")[noise_id]
        simulate(K_inf, f, x_ref, u_ref, dt, n_starting_states, noise_loaded.shape[1], noise=noise_loaded)

#### Cartpole
We want to simulate a cartpole. The dynamics for cartpole is given below.

In [ ]:
def sim_cartpole(x0, u, dt):
    DT, t = 0.1, 0
    def dynamics(x, u):
        mc, mp, l, g, I = 10.0, 1.0, 0.5, 9.81, 0.25
        s, c = np.sin(x[1]), np.cos(x[1])
        xddot = (u + mp * s *(l * np.square(x[3]) + g * c)) / (mc + mp * np.square(s))
        tddot = (-u * c - mp * l * np.square(x[3]) * c * s 
                 -(mc + mp) * g * s) / (l * (mc + mp * np.square(s)))
        return np.concatenate([x[2:4], xddot, tddot])
        
    while t < dt:
        current_dt = min(DT, dt-t)
        x0 = x0 + current_dt * dynamics(x0, u)
        t += current_dt
    
    return x0

Find the infinite horizon controller for the linearized version of the cartpole balancing problem

In [ ]:

cartpole_config = {
    'f': sim_cartpole,
    'exp_name': "Cartpole-Balancing",
    'env': None,
    'steps': 500,
    'x_ref': np.array([0.0, np.pi, 0.0, 0.0]),
    'u_ref': np.array([0.0]),
    'legend':['x', 'theta', 'xdot', 'thetadot'],
    'ss': np.array([[0, 0, 0, 10, 50],
                    [9*np.pi/10, 3*np.pi/4, np.pi/2, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]]), #ss = starting states
    'noise': 'p_b_w',
}
lqr_nonlinear(cartpole_config)

#### Helicopter
Now, we want to try the LQ controller on a Helicopter model. Some utility functions are given below.
The dynamics is given in the cell after.

In [ ]:
def q_to_euler(q):
	# returns an equivalent euler angle representation of this quaternion
	# in roll-pitch-yaw order
	mData = q
	my_epsilon = 1e-10
	# euler = np.zeros((3,1))
	euler = np.zeros(3)

	# quick conversion to Euler angles to give tilt to user
	sqw = mData[3]*mData[3]
	sqx = mData[0]*mData[0]
	sqy = mData[1]*mData[1]
	sqz = mData[2]*mData[2]

	euler[1] = np.arcsin(2.0 * (mData[3]*mData[1] - mData[0]*mData[2]))
	if (np.pi/2 - np.abs(euler[1]) > my_epsilon):
		euler[2] = np.arctan2(2.0 * (mData[0]*mData[1] + mData[3]*mData[2]),sqx - sqy - sqz + sqw)
		euler[0] = np.arctan2(2.0 * (mData[3]*mData[0] + mData[1]*mData[2]),sqw - sqx - sqy + sqz)
	else:
		# compute heading from local 'down' vector
		euler[2] = np.arctan2(2*mData[1]*mData[2] - 2*mData[0]*mData[3], 2*mData[0]*mData[2] + 2*mData[1]*mData[3])
		euler[0] = 0.0
	        
		# If facing down, reverse yaw
		if (euler[1] < 0):
			euler[2] = np.pi - euler[2]
	return euler 

def axis_angle_dynamics_update(axis_angle0, pqr_times_dt):
	q0 = quaternion_from_axis_rotation(axis_angle0);
	q1 = quat_multiply(q0, quaternion_from_axis_rotation(pqr_times_dt));
	return axis_rotation_from_quaternion(q1);

def axis_rotation_from_quaternion(q):
	rotation_angle = 2 * np.arcsin(np.linalg.norm(q[:3]))
	my_eps = 1e-6
	if(rotation_angle < my_eps):
		# a = np.zeros((3,1))
		a = np.zeros(3)
	else:
		a = q[:3]/np.linalg.norm(q[:3]) * rotation_angle
	return a

def euler_to_q(euler):

	c1 = np.cos(euler[2] * 0.5)
	c2 = np.cos(euler[1] * 0.5)
	c3 = np.cos(euler[0] * 0.5)
	s1 = np.sin(euler[2] * 0.5)
	s2 = np.sin(euler[1] * 0.5)
	s3 = np.sin(euler[0] * 0.5)

	# q = np.zeros((4,1))
	q = np.zeros(4)

	q[0] = c1*c2*s3 - s1*s2*c3
	q[1] = c1*s2*c3 + s1*c2*s3
	q[2] = s1*c2*c3 - c1*s2*s3
	q[3] = c1*c2*c3 + s1*s2*s3

	return q

def express_vector_in_quat_frame(vin, q):
	# print(q[:3])
	# print(q[3])
	return rotate_vector(vin, np.append(-q[:3], q[3]))

def quaternion_from_axis_rotation(axis_rotation):
	# pass
	# 1/0
	rotation_angle = np.linalg.norm(axis_rotation)
	# quat = np.zeros((4,1))
	quat = np.zeros(4)
	#1/0
	if (rotation_angle < 1e-4):
		quat[:3] = axis_rotation/2
	else: 
		normalized_axis = axis_rotation / rotation_angle
		quat[:3] = normalized_axis * np.sin(rotation_angle / 2)
	
	quat[3] = np.sqrt(1 - np.linalg.norm(quat[:3])**2)
	return quat

def quat_multiply(lq, rq):
	# quaternion entries in order: x, y, z, w
	# quat = np.zeros((4, 1))
	quat = np.zeros(4)
	quat[0] = lq[3]*rq[0] + lq[0]*rq[3] + lq[1]*rq[2] - lq[2]*rq[1]
	quat[1] = lq[3]*rq[1] - lq[0]*rq[2] + lq[1]*rq[3] + lq[2]*rq[0]
	quat[2] = lq[3]*rq[2] + lq[0]*rq[1] - lq[1]*rq[0] + lq[2]*rq[3]
	quat[3] = lq[3]*rq[3] - lq[0]*rq[0] - lq[1]*rq[1] - lq[2]*rq[2]
	return quat

def rotate_vector(vin, q):
	# temp = quat_multiply(q, np.concatenate([vin, 0]))
	temp = quat_multiply(q, np.append(vin, 0))
	# vout = quat_multiply(temp, np.concatenate([-q[:3], q[3]]))
	vout = quat_multiply(temp, np.append(-q[:3], q[3]))
	return vout[:3]

def rotate_vector_by_inverse_quaternion(vin, q):
	vout = quat_multiply(quat_multiply(np.append(-q[:3], q[3]), np.append(vin, 0)), q)
	return vout[:3]

In [ ]:
def sim_heli(x0, u0, total_dt):
    DT, t = 0.05, 0
    ####################################
    #set up idx, model params, features#
    ####################################
    idx, model = dict(), dict()
    k = 0
    keys = ["ned_dot", "ned", "pqr", "axis_angle"]
    for ky in range(len(keys)):
        idx[keys[ky]] = np.arange(k,k+3)
        k += 3
        
    keys = ["m", "Ixx", "Iyy", "Izz", "Tx", "Ty", "Tz", "Fx", "Fy", "Fz"]
    values = [5, .3, .3, .3, np.array([0, -1.0410, 3.9600]), 
             np.array([0, -0.9180, -2.7630]), np.array([0, -0.7740, 4.4520]),
             np.array([-0.2400]), np.array([-3.0, -0.6000]), np.array([0, -0.0025, -137.5000])]
    for ky in range(len(keys)):
        model[keys[ky]] = values[ky]
        
    def compute_forces_and_torques(x0, u0, model, idx):
        # compute helicopter velocity in its own frame 
        # (it experiences drag forces in its own frame)
        uvw = express_vector_in_quat_frame(x0[idx['ned_dot']], quaternion_from_axis_rotation(x0[idx['axis_angle']]))
        ## aerodynamic forces 

        # expressed in heli frame:
        Fxyz_minus_g = np.zeros(3)
        Fxyz_minus_g[0] = np.dot(model['Fx'], uvw[0])
        Fxyz_minus_g[1] = np.dot(model['Fy'], np.array([1, uvw[1]]))
        Fxyz_minus_g[2] = np.dot(model['Fz'], np.array([1, uvw[2], u0[3]]))

        # expressed in ned frame
        F_ned_minus_g = rotate_vector(Fxyz_minus_g, quaternion_from_axis_rotation(x0[idx['axis_angle']]))

        # add gravity to complete the forces:
        Fned = F_ned_minus_g + np.dot(model['m'], np.array([0, 0, 9.81]))

        ## torques
        Txyz = np.zeros(3)
        Txyz[0] = np.dot(model['Tx'], np.array([1, x0[idx['pqr'][0]], u0[0]]))
        Txyz[1] = np.dot(model['Ty'], np.array([1, x0[idx['pqr'][1]], u0[1]]))
        Txyz[2] = np.dot(model['Tz'], np.array([1, x0[idx['pqr'][2]], u0[2]]))
        return Fned, Txyz
        
    x1 = np.zeros_like(x0)
    while t < total_dt:
        dt = min(DT, total_dt-t)
        # compute forces and torques
        Fned, Txyz = compute_forces_and_torques(x0, u0, model, idx)
        
        # forward integrate state            
        # angular rate and velocity simulation:  [this ignores inertial coupling;
        # apparently works just fine on our helicopters]

        x1[idx['ned_dot']] = x0[idx['ned_dot']] + dt * Fned / model['m']
        x1[idx['pqr']] = x0[idx['pqr']] + dt * Txyz / np.array([model['Ixx'], model['Iyy'], model['Izz']])


        # position and orientation merely require integration (we use Euler integration):
        x1[idx['ned']] = x0[idx['ned']] + dt * x0[idx['ned_dot']]
        x1[idx['axis_angle']] = axis_angle_dynamics_update(x0[idx['axis_angle']], x0[idx['pqr']]*dt)

        x0 = x1
        t += dt
        
    return x0



Find the infinite horizon controller for the linearized version of the hovering copter

In [ ]:
# Just run the cell below to generate plots using the code you wrote for cartpole!
x_ref, u_ref = np.zeros(12), np.zeros(4)
x_ref[9] = np.arcsin(3.0/(5*9.81)) 
u_ref[3] = 9.81*5*np.cos(x_ref[9])/137.5
heli_config = {
    'f': sim_heli,
    'env': None,
    'exp_name': "Helicopter-Hovering",
    'steps': 200,
    'x_ref': x_ref,
    'u_ref': u_ref,
    'ss': loadmat("mats/p_c_heli_starting_states.mat")["heli_starting_states"], #ss = starting states
    'noise': 'p_c_w',
}
lqr_nonlinear(heli_config)

## LQ for Linear Time Varying (LTV) Systems

Now we'll consider LTV systems, i.e., $x_{t+1} = A_tx_t + B_tu_t$ (and $Q, R$ may be time-dependent). Our optimal control problem is thus:

$$\min_{x,u} \sum_{t=1}^T (x_t' Q_t x_t + u_t' R_t u_t) + x_{T+1}' Q_{T} x_{T+1} \\ \text{s.t. }x_{t+1} = A_t x_t + B_t u_t$$


Eyeball your plots to sanity check your implementation as this will be important for part (d)!

In [ ]:
# implement a finite horizon optimal feedback controller, accounting for possibly time-varying parameters
def lqr_finite_horizon(A_lst, B_lst, Q_lst, R_lst, T):
    """
    Each of A_lst, B_lst, Q_lst, and R_lst is either a python list (of length T) of numpy arrays 
        or a numpy array (indicating this parameter is not time-varying).
    You will need to handle both cases in your implementation
    
    Find the finite horizon K and P through running LQR back-ups
    return: K_{1:T}, P_{1:T}
    """
    
    K_lst, P_lst= [], []
    """YOUR CODE HERE"""
    
          
    """YOUR CODE ENDS HERE"""
    return K_lst, P_lst

Now we define a LTV system for a fixed horizon

In [ ]:
T = 100
A_lst = [np.array([[np.sin(t), -0.5049, 0.0299, 2.6544, 1.0608],
              [2.3846, -0.2312, -0.1260, -0.7945, 0.5279],
              [1.4019, -0.6394, -0.1401, 0.5484, 0.1624],
              [-0.0254, 0.4595, -0.0862, 2.1750, 1.1012],
              [0.5172, 0.5060, 1.6579, -0.9407, -1.4441]]) for t in range(T)]
B_lst = [np.array([[-0.7789, -1.2076],
              [0.4299, -1.6041],
              [0.2006, -1.7395],
              [0.8302, 0.2295],
              [-1.8465, np.cos(t)]]) for t in range(T)]

starting_states = np.array([[-1.9613, 1.9277, -0.2442],
                            [-1.3127, -0.2406, -0.0260],
                            [0.0698, -0.5860, -0.7522],
                            [0.0935, -0.1524, -0.9680],
                            [1.2494, 0.5397, -0.5146]])
n_starting_states = starting_states.shape[1]

dx, du = A_lst[0].shape[0], B_lst[0].shape[1]
Q, R = np.eye(dx), np.eye(du)
K_lst, P_lst = lqr_finite_horizon(A_lst, B_lst, Q, R, T)

# fill in to use your controller
def simulate(A_lst, B_lst, K_list, n_starting_states, T, noise=None):
    for s in np.arange(n_starting_states):
        x, u = np.zeros((K_list[0].shape[1], T+1)), np.zeros((K_list[0].shape[0], T+1))
        x[:,0] = starting_states[:,s]
        for t in np.arange(T):
            """YOUR CODE HERE"""
            u[:,t] = 
            """YOUR CODE ENDS HERE"""
            x[:,t+1] = A_lst[t] @ x[:,t] + B_lst[t] @ u[:,t]
            if noise is not None:
                x[:,t+1] += noise[:,t]
      
        plt.plot(x.T, linewidth=.7)
        plt.plot(np.squeeze(u.T), linewidth=.7, linestyle='--')
        legend_elements = [Line2D([0], [0], label='x'),Line2D([0], [0], linestyle='--', label='u')]
        plt.legend(handles=legend_elements)
        plt.xlabel('time')
        plt.title("LTV Sanity Check")
        plt.show()

# simulate to sanity check your TV solution
simulate(A_lst, B_lst, K_lst, n_starting_states, T)

#### Trajectory Following for Nonlinear Systems

Given a feasible trajectory $\{ x_t, u_t\}_{t=0}^{H-1}$ , we define our optimization problem as follows:

$$\min_{u_0, ..., u_{H-1}} \sum_{t=0}^{H-1} (((x_t-x_t^*)' Q_t (x_t-x_t^*) + (u_t-u_t^*)' R_t (u_t-u_t^*))) \\ \text{s.t. }x_{t+1} = f(x_t, u_t)$$

Your task is to implement trajectory following for helicopter flight (non-linear system) by **transforming the objective into a LTV setting and running LQR.** We have provided (and loaded) the reference trajectory below, run the following cell to visualize the target trajectory. **Note that this trajectory is *approximately feasible*, so you will have to include an offset term to account for this.** 

HINT: for the offset, refer to the lecture and now use the optional argument to the `linearize_dynamics` function. Now, what to do with these offsets? Since we have written a time-varying LQR solver for linear systems, in order to use the same code, augment the "state" to include the offset in your A and B matrices.

In [ ]:
traj = loadmat("mats/heli_traj.mat")
x_init, x_target, u_target = traj['x_init'], traj['x_target'], traj['u_target']

plt.plot(x_target.T, linewidth = .6)
plt.title("Visualization of Target Helicopter Trajectory")
plt.xlabel("time")
plt.show()

In [ ]:
f = sim_heli
dt = 0.1 # we work with discrete time

x_ref, u_ref = x_target.T, u_target.T
my_eps = 0.001 # finite difference for numerical differentiation
T, dx = x_ref.shape
du = u_ref.shape[1]
A_lst, B_lst = [], [] # this should look familiar, maybe your code from part (c) will be helpful!

for t in range(T-1):
    """YOUR CODE HERE"""
    A_t = 
    B_t = 
    """YOUR CODE ENDS HERE"""
    A_lst.append(A_t)
    B_lst.append(B_t)

Q, R = np.eye(A_lst[0].shape[0]), np.eye(B_lst[0].shape[1])
Q[-1, -1] = 0
K_list, P_list = lqr_finite_horizon(A_lst, B_lst, Q, R, T-1) # you wrote this in part (c)

# once again fill in the control input based on your controller
def simulate(K_lst, f, x_ref, u_ref, dt, n_starting_states, T, noise=None):
    def setup_heli_idx():
        idx = dict()
        k = 0
        keys = ["ned_dot", "ned", "pqr", "axis_angle"]
        for ky in range(len(keys)):
            idx[keys[ky]] = np.arange(k,k+3)
            k += 3
        return idx
    idx = setup_heli_idx()
    
    def disp(sim, ref, label):
        cp = sns.color_palette("Paired")
        a, b = sim[idx[label]], ref[idx[label]]
        [plt.plot(a[i], linewidth=1, color=cp[i]) for i in range(a.shape[0])]
        [plt.plot(b[i], linewidth=2, linestyle=':', color=cp[i]) for i in range(b.shape[0])]
        legend_elements = [Line2D([0], [0], label='yours'),Line2D([0], [0], linestyle=':', label='target')]
        plt.legend(handles=legend_elements)
        plt.xlabel('time')
        plt.title(label)
        plt.show()
        
    for s in np.arange(n_starting_states):
        x, u = np.zeros((x_ref.shape[1], T)), np.zeros((u_ref.shape[1], T))
        x[:,0] = starting_states[:,s]
        for t in np.arange(T-1):
            """YOUR CODE HERE"""
            x_aug = np.concatenate([x[:, t] - x_ref[t], [1]])
            u[:,t] = K_list[t] @ x_aug + u_ref[t]
            """YOUR CODE ENDS HERE"""
            x[:,t+1] = f(x[:,t], u[:,t], dt)
            if noise is not None:
                x[:,t+1] += noise[:,t]
        keys = ["ned_dot", "ned", "pqr", "axis_angle"]
        for key in keys:
            disp(x, x_ref.T, key)
                 
                
# simulate (reference trajectory depicted by dotted lines)               
starting_states = x_init.reshape(-1,1)      
simulate(K_list, f, x_ref, u_ref, dt, 1, T)

# now with noise!
simulate(K_list, f, x_ref, u_ref, dt, 1, T, noise = np.random.normal(scale=.1, size=(x_ref.shape[1], T)))